In [ ]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spark4ai-prediction/soil_dataset/sample_submission.csv
/kaggle/input/spark4ai-prediction/soil_dataset/train.csv
/kaggle/input/spark4ai-prediction/soil_dataset/test.csv


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
df_train = pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/train.csv')
df_test = pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/test.csv')

In [ ]:
df_train.head(10)

,Id,Property_A,Property_B,Property_C,Property_D,Property_E,Property_F,A,B,C,...,2491 nm,2492 nm,2493 nm,2494 nm,2495 nm,2496 nm,2497 nm,2498 nm,2499 nm,2500 nm
0,1,5.99,0.115797,1.638,49.0,19.0,32.0,101745.99750,292211.0199,38816.97313,...,0.385093,0.384933,0.384977,0.385273,0.385507,0.384700,0.383193,0.382067,0.381257,0.381013
1,2,5.15,0.183483,1.482,47.0,16.0,37.0,105712.64020,315908.0751,39888.72776,...,0.404990,0.405053,0.405617,0.406607,0.407490,0.406543,0.404393,0.403207,0.402917,0.402393
2,3,5.55,0.173130,1.404,50.0,20.0,30.0,102621.33090,304334.3675,42120.98104,...,0.397750,0.397417,0.397273,0.397140,0.397013,0.396677,0.396240,0.396140,0.396370,0.397213
3,4,5.09,0.196011,1.638,50.0,15.0,35.0,113446.28810,311356.2657,39195.84181,...,0.382153,0.382130,0.382580,0.383233,0.383877,0.383550,0.382607,0.382263,0.382473,0.382533
4,5,4.73,0.203319,1.677,53.0,19.0,28.0,109714.60330,322110.4501,38347.65382,...,0.416157,0.416683,0.417567,0.417703,0.417420,0.416463,0.415003,0.414080,0.413637,0.413687
5,6,4.75,0.243078,1.755,48.0,20.0,32.0,91440.09771,306100.1273,38980.80823,...,0.433190,0.433410,0.434183,0.434707,0.434990,0.434143,0.432543,0.431837,0.431957,0.432500
6,7,5.04,0.201579,1.638,53.0,19.0,28.0,107607.66030,311366.1395,40291.48906,...,0.406410,0.406193,0.405853,0.405113,0.404200,0.403337,0.402570,0.402697,0.403703,0.405377
7,8,5.21,0.247863,1.794,45.0,17.0,38.0,101776.71090,288458.9862,38764.06804,...,0.378983,0.379943,0.381267,0.381450,0.380993,0.379630,0.377633,0.376960,0.377580,0.377690
8,9,5.19,0.175305,1.677,50.0,15.0,35.0,103416.80940,278716.8635,37147.90302,...,0.341433,0.341167,0.340957,0.340717,0.340417,0.339770,0.338950,0.338827,0.339363,0.340823
9,10,5.17,0.198882,1.521,49.0,16.0,35.0,109283.07930,304174.7415,39858.00868,...,0.367893,0.367960,0.368313,0.368577,0.368680,0.367923,0.366623,0.366247,0.366737,0.367930


In [ ]:
# I first checked for any empty cell
#emt_col store the number of cells in each row
emt_col = df_train.isnull().sum()

#Then I calculated tot_val for the total empty cells in the data
tot_val = emt_col.sum()

print(emt_col)
print(tot_val)

Id            0
Property_A    0
Property_B    0
Property_C    0
Property_D    0
             ..
2496 nm       0
2497 nm       0
2498 nm       0
2499 nm       0
2500 nm       0
Length: 2174, dtype: int64
0


In [ ]:
'''Since the total value is 0 there is no need to fill any cell as there are no empty cell
After this, there were a lot a features from sensor 2 and their values were kinda of
same(from ovservation) so then I checked for correlation between the features and removed
the features which were highly correlated'''

In [ ]:
# X = df_train.iloc[:,7:]
# corr_matrix = X.corr()

# #defining the threshold. I initially took 0.8 then I took 0.9 (gave the same result)
# corr_thresh = 0.9

# #finding and storing the correlated features
# corr_ftrs = set()
# for i in range(len(corr_matrix.columns)):
#     for j in range(i):
#         if abs(corr_matrix.iloc[i, j]) >= corr_thresh:
#             colname = corr_matrix.columns[i]
#             corr_ftrs.add(colname)

# #Then I removed the correlated features
# X = X.drop(columns=corr_ftrs)


In [ ]:
from sklearn.decomposition import PCA

X = df_train.iloc[:,7:]
pca = PCA(n_components=17)
pca.fit(X)
X = pca.transform(X)

In [ ]:
#From this we can say that all the features from sensor2 were highly correlated

In [ ]:
#taking all the targets in y
y = df_train.iloc[:,1:7]
print(y)

     Property_A  Property_B  Property_C  Property_D  Property_E  Property_F
0          5.99    0.115797       1.638        49.0        19.0        32.0
1          5.15    0.183483       1.482        47.0        16.0        37.0
2          5.55    0.173130       1.404        50.0        20.0        30.0
3          5.09    0.196011       1.638        50.0        15.0        35.0
4          4.73    0.203319       1.677        53.0        19.0        28.0
..          ...         ...         ...         ...         ...         ...
245        5.10    1.550000       0.980         0.5        57.1        42.2
246        4.90    3.090000       1.060         5.1        59.6        35.3
247        4.90    0.620000       0.860         0.1        57.5        42.4
248        5.10    2.960000       0.850         1.9        68.8        29.3
249        5.40    2.060000       1.010         0.7        64.8        34.5

[250 rows x 6 columns]


In [ ]:
#split the data into train and test to find the RMSE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#Tried to apply basic ensemble using average of three models
#Trained the model and saved the predictions

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
prediction1 = model.predict(X_test)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
prediction2 = model.predict(X_test)

In [ ]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)
prediction3 = model.predict(X_test)

In [ ]:
predictions = (prediction1 + prediction2 + prediction3)/3

In [ ]:
rmse = sqrt(mean_squared_error(y_test, predictions))
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
#RMSE is acceptable so now I trained all the models on the whole data without splitting in test/train

In [ ]:
X_train = X
y_train = y

In [ ]:
df_test

,Id,A,B,C,D,E,F,G,H,I,...,2491 nm,2492 nm,2493 nm,2494 nm,2495 nm,2496 nm,2497 nm,2498 nm,2499 nm,2500 nm
0,1,111184.24210,346946.2804,43356.57078,5164.670838,163.384072,109.192843,461.122552,45783.17392,49.689130,...,0.390053,0.390647,0.391547,0.391620,0.391353,0.391007,0.390543,0.390117,0.389720,0.389410
1,2,105506.86000,309880.1367,44501.70988,5223.016492,173.993427,94.721502,317.640213,50907.18706,43.586957,...,0.394680,0.395323,0.396040,0.395467,0.394440,0.393930,0.393730,0.393800,0.394157,0.395083
2,3,113968.41680,336280.9600,44737.22285,5121.451835,169.749685,101.299384,266.184478,46709.86895,49.689130,...,0.396113,0.395733,0.395470,0.394950,0.394390,0.394030,0.393823,0.393937,0.394363,0.395190
3,4,89953.56650,361447.5617,38306.69505,4831.884516,146.409103,74.987856,271.132144,33103.54427,28.767391,...,0.446693,0.446783,0.447223,0.447357,0.447360,0.447080,0.446593,0.446193,0.445857,0.445790
4,5,107285.16910,319862.5212,45049.53351,5219.775066,163.384072,115.770725,307.744879,48222.54600,44.458696,...,0.349563,0.348673,0.348093,0.348110,0.348467,0.348823,0.349217,0.349270,0.348963,0.349193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,86544.37301,326991.3852,30241.22945,4926.966322,167.627814,68.409974,648.144360,24916.44335,25.280435,...,0.316780,0.315620,0.314637,0.314143,0.313890,0.313427,0.312940,0.313147,0.314017,0.315663
97,98,97208.08453,355238.6041,28524.37410,5143.061336,163.384072,82.881314,628.353692,26068.43071,30.510870,...,0.280483,0.279843,0.279640,0.279717,0.279877,0.279400,0.278517,0.277797,0.277193,0.277050
98,99,79851.91120,349803.0921,31106.48359,5743.805476,135.799748,56.569786,398.781950,22224.03359,22.665217,...,0.294737,0.294613,0.294897,0.295330,0.295707,0.295100,0.293843,0.293060,0.292703,0.292727
99,100,98562.54788,359232.5453,26618.08441,5051.220955,135.799748,89.459196,527.421288,26646.64402,32.254348,...,0.297890,0.297577,0.297727,0.298143,0.298627,0.298403,0.297723,0.297373,0.297323,0.297500


In [ ]:
#checking for null values in test dataset
emt_col = df_test.isnull().sum()
tot_val = emt_col.sum()

print(emt_col)
print(tot_val)

Id         0
A          0
B          0
C          0
D          0
          ..
2496 nm    0
2497 nm    0
2498 nm    0
2499 nm    0
2500 nm    0
Length: 2168, dtype: int64
0


In [ ]:
#changing the test data according to the training data
X_test = df_test.iloc[:,1:18]

In [ ]:
#trained the models on the whole data and saved the predictions of test data

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
prediction1 = model.predict(X_test)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
prediction2 = model.predict(X_test)

In [ ]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)
prediction3 = model.predict(X_test)

In [ ]:
predictions = (prediction1 + prediction2 + prediction3)/3

In [ ]:
predictions

array([[  5.55694173,   2.00749771,   1.13531873,  24.82134861,
         48.73145244,  30.61536705],
       [  5.28925383,   2.03126788,   1.22187878,  30.1942328 ,
         36.12166227,  34.76823388],
       [  5.25255606,   2.28048466,   1.09315003,  22.96508805,
         46.09293902,  31.4535484 ],
       [  5.61658247,   2.09820341,   1.20388072,  33.65063042,
         34.95290423,  23.13713141],
       [  5.45112478,   1.97541761,   0.96776137,   8.69226749,
         57.02809833,  35.40601299],
       [  5.4369739 ,   1.79115951,   1.01335312,  19.11991052,
         52.69032463,  30.68909513],
       [  5.53775036,   2.6951358 ,   1.02651784,   6.05360554,
         54.06422887,  36.0690814 ],
       [  5.40547366,   2.2629431 ,   1.13375952,  10.24608491,
         55.89294116,  33.37065778],
       [  5.39229515,   1.56532507,   1.0284322 ,  16.94167551,
         52.06181871,  33.04353278],
       [  5.24096046,   2.90669693,   0.87934169,   2.2493482 ,
         62.1062556 ,  34.3

In [ ]:
final_prediction = np.sum(predictions, axis = 1)
final_prediction

array([112.86792628, 109.62652944, 109.13776622, 100.65933265,
       109.52068257, 110.74081681, 105.4463198 , 108.31186013,
       110.03307943, 107.71999901, 108.82116829, 108.1058935 ,
       108.25590692, 112.64032619, 109.32456161, 108.18022157,
       100.26615977, 106.92891648, 108.03090549, 109.70175194,
       103.4655849 , 125.95985122, 108.97788733, 108.6983874 ,
       107.44841959, 112.97815444, 108.39270031, 107.85706311,
       101.180179  , 110.58460448, 101.40186859, 107.91573704,
       108.90645378, 116.35199469, 105.89077593, 110.44625682,
       116.03920839, 105.04770512, 105.69293861, 108.82617563,
       108.11468667, 103.50371626, 105.44721474, 107.00185642,
       109.0091272 , 108.09033897, 106.62604028, 116.36827006,
       103.01859035, 109.96206928, 123.03823502,  99.41035744,
       106.58619556, 104.05908485, 102.78090688, 108.41293923,
       112.54021672, 112.01362667, 107.47205302, 101.39020391,
        91.32428693, 116.79695629, 116.28885535, 101.15

In [ ]:
submission = pd.DataFrame({'Id': df_test['Id'], 'Predicted': final_prediction})

In [ ]:
submission

,Id,Predicted
0,1,112.867926
1,2,109.626529
2,3,109.137766
3,4,100.659333
4,5,109.520683
...,...,...
96,97,105.867594
97,98,102.989264
98,99,100.365729
99,100,103.283491


In [ ]:
submission.to_csv('submission.csv', index=False)